In [1]:
#!pip install dash-bootstrap-components
#!pip install jupyter-dash

In [1]:
from jupyter_dash import JupyterDash 
from dash import dcc
from dash import html
from dash.dependencies import Input, Output 
import plotly.express as px
import dash_bootstrap_components as dbc
import pandas as pd
#import pandas_datareader.data as web #it is an in-built pandas API to read data from the web
#import datetime

# Load data

data is for now stored on my local machine. need a script to load and generate it from raw data

In [ ]:
%run 0_download_data.ipynb

In [3]:
%run 1_clean_data_backbone.ipynb

  0%|          | 0/4 [00:00<?, ?it/s]

Loading data...


 25%|██▌       | 1/4 [00:11<00:35, 11.72s/it]

Loading data...


 50%|█████     | 2/4 [00:20<00:19,  9.96s/it]

Loading data...


 75%|███████▌  | 3/4 [00:28<00:09,  9.11s/it]

Loading data...


100%|██████████| 4/4 [00:36<00:00,  9.25s/it]


Finished...
Concatenate...
Finished...
Doesn't exists


In [2]:
clean_files = ['../data/cleaned_data/all_dataset.parquet']
dfs = []
for file in clean_files:
    dfs.append(pd.read_parquet(file))
df = pd.concat(dfs, axis = 0)

df.index = pd.to_datetime(df.index)
df['state_of_charge_percent'] = df['state_of_charge_percent'].clip(0,120)

In [5]:
df['month_name'] = df.index.month_name()

In [82]:
import numpy as np
field = 'current'

def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

df['day_of_week'] = df.index.day_of_week
df.groupby(['battery_id','day_of_week'])[field].agg(['mean', percentile(25), percentile(75)])

mean  percentile_25  percentile_75
battery_id day_of_week                                        
0          0            0.025022        -0.1460          0.580
           1            0.007561        -0.0920          0.582
           2           -0.001185        -0.2050          0.584
           3           -0.037193        -0.1980          0.580
           4            0.004701        -0.1120          0.582
...                          ...            ...            ...
199        2           -0.009301        -0.2950          0.412
           3           -0.019575        -0.1010          0.406
           4           -0.009674        -0.1805          0.406
           5            0.006890        -0.1655          0.424
           6           -0.016399        -0.2115          0.410

[1400 rows x 3 columns]

# Set ut dash app

In [22]:
#Building the app
app = JupyterDash(__name__, external_stylesheets = [dbc.themes.SKETCHY],
                            meta_tags=[{'name': 'viewport',
                            'content': 'width=device-width, initial-scale=1.0'}]) #if you want to make it adjust when the dashboard is visualized on mobile phones

In [27]:
list_months = ['Jan', 'Mar', 'Jun']
option_resampling = [{'label':'month','value':'1MS'},{'label':'day','value':'1D'}]

In [43]:
#Step 1: create app
app = JupyterDash(__name__)

#Step 2: create layout
app.layout = html.Div([
                       html.H1('TS try'),
                       dcc.Graph(id= 'ts-graph'),
                       dcc.Dropdown(id = 'id-dropdown',
                                    value = '1',
                                    options = [{'label': c, 'value': c} for c in df['battery_id'].unique()]),
                       dcc.Dropdown(id = 'var-dropdown',
                                    value = 'voltage',
                                    options = [{'label': a, 'value': a} for a in df.columns]),
                       dcc.Dropdown(id = 'resampling-dropdown',
                                    value = '1MS',
                                    options = option_resampling)
])

#Step 4: callback
#https://stackoverflow.com/questions/69725119/dash-callback-with-2-input
@app.callback(
    Output(component_id = 'ts-graph', component_property = 'figure'),
    [
    Input(component_id = 'id-dropdown', component_property = 'value'),
    Input(component_id = 'var-dropdown', component_property = 'value'),
    Input(component_id = 'resampling-dropdown', component_property = 'value'),
    ]
)

#Step 3: plotting function
def update_figure(selected_id, selected_var, resampling):
    df_plot = df[df['battery_id'] == selected_id]
    df_plot = df_plot.resample(resampling).mean()
    line_fig = px.line(df_plot.reset_index(), x = 'timestamp', y = selected_var)
    line_fig.show()
    return line_fig

In [44]:
if __name__ == '__main__':
  app.run_server(debug = True)

Dash app running on http://127.0.0.1:8050/


C:\Users\PC_victor\Anaconda3\lib\site-packages\jupyter_dash\jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



In [5]:
#Building the layout
app.layout = dbc.Container([
                            #Row 1: title + subtitle
                            dbc.Row([
                                     dbc.Col([html.H1('BBoxx Lithium-Ion Battery Field Data Challenge', className = 'text-center mb-4')], width = 12),
                                     dbc.Col([html.P('Team 22: Victor Bossard, Etienne Beauchamp, Elias Galiounas, Maria Varini', className = 'text-center mb-4')], width = 12),
                            ]), #close Row 1

                            #Row 2: Data visualization
                            dbc.Row([
                                     #Title of the section
                                     dbc.Col([html.H1('Data visualization', className = 'text-primary text-left mb-4')], width = 12),
                                     #Column 1 (left): Dropdown list + time-series graph
                                     dbc.Col([dcc.Dropdown(id = 't_s-dropdown', multi = False, value = 'Jan', options = [{'label':x, 'value': x} for x in list_months]),
                                              dcc.Graph(id = 't_s-graph', figure = {})      
                                     ], width = 5), #close Column 1 (Row 2)
                                     #Column 2 (right): Dropdown list + statistic graph
                                     dbc.Col([dcc.Dropdown(id = 'stat-dropdown', multi = False, value = 'Jan', options = [{'label':x, 'value': x} for x in list_months]),
                                              dcc.Graph(id = 'stat-graph', figure = {})      
                                     ], width = 5), #close Column 2 (Row 2)
                            ]), # Close Row #2

                            #Row 3: Cluster results visualization
                            dbc.Row([
                                     #Title of the section
                                     dbc.Col([html.H1('Cluster results visualization', className = 'text-left mb-4')], width = 12),
                                     #Column 1 (left): Dropdown list + clusering results graph
                                     dbc.Col([dcc.Dropdown(id = 'cluster-dropdown', multi = False, value = 'Jan', options = [{'label':x, 'value': x} for x in list_months]),
                                              dcc.Graph(id = 'cluster-graph', figure = {})      
                                     ], width = 5), #close Column 1 (Row 3)
                                     #Column 2 (right): Dropdown list + statistic graph
                                     dbc.Col([dcc.Dropdown(id = 'stat_clust-dropdown', multi = False, value = 'Jan', options = [{'label':x, 'value': x} for x in list_months]),
                                              dcc.Graph(id = 'stat_clust-graph', figure = {})      
                                     ], width = 5), #close Column 2 (Row 3)
                            ]), # Close Row #3

                            #Row 4: Extracting user profiles
                            dbc.Row([
                                     #Title of the section
                                     dbc.Col([html.H1('Extract user profiles', className = 'text-left mb-4')], width = 12),
                                     #Column 1 (left): Dropdown list + K means graph
                                     dbc.Col([dcc.Dropdown(id = 'k-dropdown', multi = False, value = 'Jan', options = [{'label':x, 'value': x} for x in list_months]),
                                              dcc.Graph(id = 'k-graph', figure = {})      
                                     ], width = 7), #close Column 1 (Row 4)
                                     #Column 2 (right): Score card placeholder
                                     dbc.Col([html.P('Score card placeholder', className = 'text-left mb-4')], width = 3),
                            ])
], fluid = True) #Close the whole dbc container. 